# Algoritmo de Grover para Encontrar el Valor Mínimo en un Arreglo No Ordenado

Este código es un Jupyter Notebook que implementa el algoritmo de Grover para buscar en un arreglo no ordenado de números.

1. La primera celda importa las librerías necesarias, incluyendo NumPy y Qiskit.
2. La segunda celda define dos funciones: `find_min` y `grover_search`. La función `find_min` utiliza la función `grover_search` para encontrar el valor mínimo en un arreglo no ordenado. La función `grover_search` implementa el algoritmo de Grover, que es un algoritmo cuántico para buscar en un arreglo no ordenado.
3. La tercera celda crea un arreglo no ordenado de 16 números y lo mezcla aleatoriamente. Luego llama a la función `find_min` con este arreglo y 64 iteraciones para encontrar el valor mínimo.

El algoritmo de Grover es un algoritmo cuántico que puede buscar en un arreglo no ordenado en tiempo O(sqrt(N)), donde N es el tamaño del arreglo. Esto es mucho más rápido que los algoritmos clásicos para buscar en un arreglo no ordenado, que tienen una complejidad temporal de O(N).

En este código, la función `grover_search` utiliza la implementación del algoritmo de Grover de Qiskit para encontrar el índice del valor mínimo en el arreglo. La función primero crea un circuito oráculo que marca los índices de los elementos en el arreglo que son menores o iguales a un umbral dado. Luego aplica el operador de Grover a este circuito oráculo, lo que amplifica la probabilidad de encontrar el valor mínimo. Finalmente, mide la salida del circuito cuántico y devuelve el índice del valor mínimo.

En resumen, este código demuestra cómo implementar el algoritmo de Grover usando Qiskit y Python.

## Versiones usadas en este cuaderno
 - Python 3.12.1
 - qiskit 2.1.0
 - qiskit-aer 0.17.1

In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.circuit.library import grover_operator 

import random

In [2]:
# numbers = unsorted array of numbers
def find_min(numbers, iterations):
    n_items = len(numbers)
    threshold = numbers[random.randint(0, n_items-1)] # Choosing a random threshold value
    candidate_indices = [i for i in range(n_items) if numbers[i] <= threshold] # Marking elements of numbers that are <= threshold
    for _ in range(iterations):
        new_threshold = numbers[grover_search(candidate_indices, n_items)] # Grover search to find element smaller than current threshold
        if new_threshold < threshold:
            threshold = new_threshold
            candidate_indices = [idx for idx in candidate_indices if numbers[idx] <= threshold]
    return threshold

def grover_search(candidate_indices, n_items):
    n_qubits = int(np.ceil(np.log2(n_items)))
    oracle = np.identity(2**n_qubits)
    for idx in candidate_indices:
        oracle[idx, idx] = -1 # Building oracle
    oracle_circuit = QuantumCircuit(n_qubits)
    oracle_circuit.unitary(oracle, range(n_qubits))
    grover_op = grover_operator(oracle_circuit, insert_barriers=True)
    grover_circuit = QuantumCircuit(n_qubits, n_qubits)
    grover_circuit.h(range(n_qubits)) # Initializing equal superposition
    n_grover_iterations = int(np.sqrt(n_items))
    for _ in range(n_grover_iterations):
        grover_circuit = grover_circuit.compose(grover_op)
    grover_circuit.measure(range(n_qubits), range(n_qubits))
    simulator = AerSimulator()
    job = simulator.run(grover_circuit)
    result = job.result()
    counts = result.get_counts()
    measured = max(counts, key=counts.get)
    return int(measured, 2)

In [13]:

unsorted_numbers = list(range(16))
unsorted_numbers.reverse()
unsorted_numbers.remove(0)  # Remove zero to avoid trivial case
unsorted_numbers.append(10)  # Append a number to ensure the list is not empty
random.shuffle(unsorted_numbers)  # Shuffle the list to create an unsorted array
print(unsorted_numbers)
find_min(unsorted_numbers, 64)

[11, 10, 10, 7, 8, 13, 4, 12, 5, 3, 14, 15, 9, 6, 2, 1]


12